In [3]:
import os

import pandas as pd

In [7]:
# result_dir = 'result-original_jet-flavor'
# date = '20250618-095124'

models = ['CNN_Baseline', 'CNN_Light', 'ParT_Baseline', 'ParT_Medium', 'ParT_Light', 'ParT_SuperLight', 'ParT_ExtremeLight']

print('- Model: $(\\eta, N_\\text{step}, N_\\text{accum}) \\Rightarrow (ACC_\\text{test}, AUC_\\text{test}, E_\\text{early})$')

for model in models:
    for date, suffix, result_dir in [
        # ('20250616-212748', 'in_decay', 'result-original_jet-flavor'),
        # ('20250617-092802', 'in_decay', 'result-original_jet-flavor'),
        # ('20250630-191524', 'ex_decay', 'result-ex_decay_jet-flavor'),

        ('20250701-010552', 'no_aug     ', 'result-ex-decay_jet-flavor'),
        ('20250701-010552', 'aug_uni_5  ', 'result-ex-decay_jet-flavor_aug_uni_5'),
        ('20250701-010552', 'aug_uni_10 ', 'result-ex-decay_jet-flavor_aug_uni_10'),
        ('20250701-010552', 'aug_rand_5 ', 'result-ex-decay_jet-flavor_aug_rand_5'),
        ('20250701-010552', 'aug_rand_10', 'result-ex-decay_jet-flavor_aug_rand_10'),

        ('20250701-010552', 'cop-no_aug     ', 'result-ex-decay_cop_jet-flavor'),
        ('20250701-010552', 'cop-aug_uni_5  ', 'result-ex-decay_cop_jet-flavor_aug_uni_5'),
        ('20250701-010552', 'cop-aug_uni_10 ', 'result-ex-decay_cop_jet-flavor_aug_uni_10'),
        ('20250701-010552', 'cop-aug_rand_5 ', 'result-ex-decay_cop_jet-flavor_aug_rand_5'),
        ('20250701-010552', 'cop-aug_rand_10', 'result-ex-decay_cop_jet-flavor_aug_rand_10'),

        # ('20250629-140436', 'aug_uni_5  ', 'result-cop_jet-flavor_aug_uni_5'),
        # ('20250630-003624', 'aug_uni_10 ', 'result-cop_jet-flavor_aug_uni_10'),
        # ('20250629-210815', 'aug_rand_5 ', 'result-cop_jet-flavor_aug_rand_5'),
        # ('20250629-025748', 'aug_rand_10', 'result-cop_jet-flavor_aug_rand_10'),
    ]:
        # Select the checkpoint directory based on the date and model.
        versions = os.listdir(os.path.join(result_dir, model))
        try:
            ckpt_dir = [v for v in versions if date in v][0]
        except IndexError:
            continue

        
        # Hyperparameters of the training.
        _, lr, batch_size, seed = ckpt_dir.split('_')

        # Read the metrics CSV file of the training result.
        df = pd.read_csv(os.path.join(result_dir, model, ckpt_dir, 'metrics.csv'))
        test_metrics = df.tail(1)

        test_accuracy = test_metrics['test_accuracy'].item()
        test_auc = test_metrics['test_auc'].item()
        early_stopped_epochs = test_metrics['epoch'].item()
        # print(f"  - {model}: ({lr}, {batch_size}) $\\Rightarrow$ ({test_accuracy:.3f}, {test_auc:.3f}, {early_stopped_epochs}) # ({date}) {suffix}")
        print(f"  - {model}: ({lr}, {batch_size}) $\\Rightarrow$ ({test_accuracy:.3f}, {test_auc:.3f}, {early_stopped_epochs}) # {suffix}")

- Model: $(\eta, N_\text{step}, N_\text{accum}) \Rightarrow (ACC_\text{test}, AUC_\text{test}, E_\text{early})$
  - CNN_Baseline: (lr1e-05, b64x8) $\Rightarrow$ (0.673, 0.729, 47) # no_aug     
  - CNN_Baseline: (lr1e-05, b64x8) $\Rightarrow$ (0.679, 0.743, 29) # aug_uni_5  
  - CNN_Baseline: (lr1e-05, b64x8) $\Rightarrow$ (0.688, 0.749, 26) # aug_uni_10 
  - CNN_Baseline: (lr1e-05, b64x8) $\Rightarrow$ (0.686, 0.751, 27) # aug_rand_5 
  - CNN_Baseline: (lr1e-05, b64x8) $\Rightarrow$ (0.674, 0.740, 25) # aug_rand_10
  - CNN_Baseline: (lr1e-05, b64x8) $\Rightarrow$ (0.658, 0.719, 43) # cop-no_aug     
  - CNN_Baseline: (lr1e-05, b64x8) $\Rightarrow$ (0.676, 0.730, 28) # cop-aug_uni_5  
  - CNN_Baseline: (lr1e-05, b64x8) $\Rightarrow$ (0.674, 0.734, 27) # cop-aug_uni_10 
  - CNN_Baseline: (lr1e-05, b64x8) $\Rightarrow$ (0.653, 0.702, 29) # cop-aug_rand_5 
  - CNN_Baseline: (lr1e-05, b64x8) $\Rightarrow$ (0.674, 0.728, 25) # cop-aug_rand_10
  - CNN_Light: (lr5e-04, b64x8) $\Rightarrow$ (0